In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/My Drive/segnet/"
os.chdir(path)
os.listdir(path)

['lab2_test_data.h5',
 'lab2_train_data.h5',
 'vgg16_bn-6c64b313.pth',
 'main_segnet.ipynb',
 'weights']

In [ ]:
import  os
print (os.getcwd()) #获取当前工作目录路径

/content/drive/My Drive/segnet


# 0. parameters

In [ ]:
import torch.utils.data as data
import torch
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
# import torch.utils.data as D
# import torchvision
# from torchvision import transforms as T

import cv2 as cv
from PIL import Image
import time

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
n_epochs = 3 # training epochs
class_num = 34
batch_size = 4
learning_rate = 2e-4
weight_decay = 5e-4
log_interval = 10
random_seed = 42
val_percent = 0.1 # training set : validation set = 9:1
torch.manual_seed(random_seed)
bn_momentum = 0.1  # BN层的momentum

cate_weight = [1/34]*34 # 损失函数中类别的权重
dir_pre_train_weights = "vgg16_bn-6c64b313.pth" # 编码器预训练权重路径
dir_weights = "./weights"
dir_checkpoint = './checkpoints'


In [ ]:
# with open(r"./a.txt","w") as f:
#    f.write('hello world')

# 1.Implement a data loader class to handle the downloaded data. (5 points)
For more information on the dataset please refer to: CityScapes dataset. 

In [ ]:
# 'rgb' stores the raw images, while 'seg' stores segmentation maps
class DataFromH5File(data.Dataset):
    def __init__(self, filepath):
        h5File = h5py.File(filepath, 'r')
        self.color_codes = h5File['color_codes']
        self.rgb = h5File['rgb']
        self.seg = h5File['seg']
        
    def __getitem__(self, idx):
        label = torch.from_numpy(self.seg[idx]).float()
        data = torch.from_numpy(self.rgb[idx]).float()
        # data = torch.from_numpy(cv.resize(self.rgb[idx], (224, 224))).float()
        data = data/255.0 # 归一化输入
        data = data.permute(2,0,1) # 将图片的维度转换成网络输入的维度（channel, width, height）
        return data, label
    
    def __len__(self):
        assert self.rgb.shape[0] == self.seg.shape[0], "Wrong data length" # 增强鲁棒性
        return self.rgb.shape[0]

In [ ]:
# load training data from lab2_train_data.h5
dataset = DataFromH5File("lab2_train_data.h5")
n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val

# split train&val
train, val = data.random_split(dataset, [n_train, n_val])
train_loader = data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last = True)
val_loader = data.DataLoader(dataset=val, batch_size=batch_size, shuffle=False, pin_memory=True, drop_last = True) # drop_last=True

# load testing data from lab2_test_data.h5
testset = DataFromH5File("lab2_test_data.h5")
test_loader = data.DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
# test the data loader
for step, (x, y) in enumerate(train_loader):
    print(x.min(),x.max())
    print(y.min(),y.max())
    print(step)
    break

print(len(train_loader), len(val_loader), len(test_loader)) # 669 74 125 when batch_size==4

tensor(0.) tensor(1.)
tensor(1.) tensor(33.)
0
669 74 125


# 2. Define the model. Provide a schematic of your architecture depicting its overall structure and the relevant parameters. (20 points)

In [ ]:
# 编码器
class Encoder(nn.Module):
    def __init__(self, input_channels):
        super(Encoder, self).__init__()

        self.enco1 = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, momentum=bn_momentum),
            nn.ReLU()
        )
        self.enco2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, momentum=bn_momentum),
            nn.ReLU()
        )
        self.enco3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=bn_momentum),
            nn.ReLU()
        )
        self.enco4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU()
        )
        self.enco5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU()
        )

    def forward(self, x):
        id = []

        x = self.enco1(x)
        x, id1 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)  # 保留最大值的位置
        id.append(id1)
        x = self.enco2(x)
        x, id2 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)
        id.append(id2)
        x = self.enco3(x)
        x, id3 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)
        id.append(id3)
        x = self.enco4(x)
        x, id4 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)
        id.append(id4)
        x = self.enco5(x)
        x, id5 = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)
        id.append(id5)

        return x, id


# 编码器+解码器
class SegNet(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(SegNet, self).__init__()

        self.weights_new = self.state_dict()
        self.encoder = Encoder(input_channels)

        self.deco1 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU()
        )
        self.deco2 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=bn_momentum),
            nn.ReLU()
        )
        self.deco3 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, momentum=bn_momentum),
            nn.ReLU()
        )
        self.deco4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, momentum=bn_momentum),
            nn.ReLU()
        )
        self.deco5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, momentum=bn_momentum),
            nn.ReLU(),
            nn.Conv2d(64, output_channels, kernel_size=3, stride=1, padding=1),
        )

    def forward(self, x):
        x, id = self.encoder(x)

        x = F.max_unpool2d(x, id[4], kernel_size=2, stride=2)
        x = self.deco1(x)
        x = F.max_unpool2d(x, id[3], kernel_size=2, stride=2)
        x = self.deco2(x)
        x = F.max_unpool2d(x, id[2], kernel_size=2, stride=2)
        x = self.deco3(x)
        x = F.max_unpool2d(x, id[1], kernel_size=2, stride=2)
        x = self.deco4(x)
        x = F.max_unpool2d(x, id[0], kernel_size=2, stride=2)
        x = self.deco5(x)

        return x

    # 删掉VGG-16后面三个全连接层的权重
    def load_weights(self, weights_path):
        weights = torch.load(weights_path)
        del weights["classifier.0.weight"]
        del weights["classifier.0.bias"]
        del weights["classifier.3.weight"]
        del weights["classifier.3.bias"]
        del weights["classifier.6.weight"]
        del weights["classifier.6.bias"]

        names = []
        for key, value in self.encoder.state_dict().items():
            if "num_batches_tracked" in key:
                continue
            names.append(key)

        for name, dict in zip(names, weights.items()):
            self.weights_new[name] = dict[1]

        self.encoder.load_state_dict(self.weights_new)

In [ ]:
from torchsummary import summary

In [ ]:
# !pip install torch-summary==1.4.4

In [ ]:
model = SegNet(input_channels=3, output_channels=class_num) # RGB images so the input_channels=3
model = model.to(device)
x = torch.ones([batch_size, 3, 128, 256]) # input shape
x = x.to(device)
y = model(x)
print(y.shape)
summary(model, input_size=(3, 128, 256))

torch.Size([4, 34, 128, 256])
Layer (type:depth-idx)                   Param #
├─Encoder: 1-1                           --
|    └─Sequential: 2-1                   --
|    |    └─Conv2d: 3-1                  1,792
|    |    └─BatchNorm2d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  36,928
|    |    └─BatchNorm2d: 3-5             128
|    |    └─ReLU: 3-6                    --
|    └─Sequential: 2-2                   --
|    |    └─Conv2d: 3-7                  73,856
|    |    └─BatchNorm2d: 3-8             256
|    |    └─ReLU: 3-9                    --
|    |    └─Conv2d: 3-10                 147,584
|    |    └─BatchNorm2d: 3-11            256
|    |    └─ReLU: 3-12                   --
|    └─Sequential: 2-3                   --
|    |    └─Conv2d: 3-13                 295,168
|    |    └─BatchNorm2d: 3-14            512
|    |    └─ReLU: 3-15                   --
|    |    └─Conv2d: 3-16                 590,080
|    |    

Layer (type:depth-idx)                   Param #
├─Encoder: 1-1                           --
|    └─Sequential: 2-1                   --
|    |    └─Conv2d: 3-1                  1,792
|    |    └─BatchNorm2d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  36,928
|    |    └─BatchNorm2d: 3-5             128
|    |    └─ReLU: 3-6                    --
|    └─Sequential: 2-2                   --
|    |    └─Conv2d: 3-7                  73,856
|    |    └─BatchNorm2d: 3-8             256
|    |    └─ReLU: 3-9                    --
|    |    └─Conv2d: 3-10                 147,584
|    |    └─BatchNorm2d: 3-11            256
|    |    └─ReLU: 3-12                   --
|    └─Sequential: 2-3                   --
|    |    └─Conv2d: 3-13                 295,168
|    |    └─BatchNorm2d: 3-14            512
|    |    └─ReLU: 3-15                   --
|    |    └─Conv2d: 3-16                 590,080
|    |    └─BatchNorm2d: 3-17           

# 3. Define the loss function and optimizer. (10 points)

In [ ]:
import torch.optim as optim

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
# cross entropy loss
# criterion = nn.CrossEntropyLoss()
# loss function
criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(np.array(cate_weight)).float()).cuda()

# 4. Train the network. (5 points)

In [ ]:
train_losses = []
train_counter = []
val_losses = []
val_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
model.load_weights(dir_pre_train_weights)


In [ ]:
def validate(epoch):
  val_pbar = tqdm(val_loader)
  for batch_idx, (data, target) in enumerate(val_pbar):
    data = data.to(device)
    target = target.to(device)
    output = model(data)
    target = target.squeeze().long()
    loss = criterion(output, target)
    val_pbar.set_description(f"    Epoch: {epoch} | Validation Loss: {loss.item():.4f}")
    if batch_idx % log_interval == 0:
        val_losses.append(loss.item())
        val_counter.append((batch_idx*batch_size) + ((epoch-1)*len(val_loader.dataset)))

In [ ]:
def train(epoch):
  model.train()
  pbar = tqdm(train_loader)
  for batch_idx, (data, target) in enumerate(pbar):
    data = data.to(device)
    target = target.to(device)
    optimizer.zero_grad()
    output = model(data)
    target = target.squeeze().long()
    # print('output shape=',output.shape)
    # print('target shape=',target.shape)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    pbar.set_description(f"Epoch: {epoch} | Loss: {loss.item():.4f}")
    if batch_idx % log_interval == 0:
        train_losses.append(loss.item())
        train_counter.append((batch_idx*batch_size) + ((epoch-1)*len(train_loader.dataset)))
        # save the parameters
        torch.save(model.state_dict(), '/content/drive/My Drive/segnet/weights/'+str(epoch)+'_model.pth')
        torch.save(optimizer.state_dict(), '/content/drive/My Drive/segnet/weights/'+str(epoch)+'_optimizer.pth')

    # validation
    if epoch % 4 == 0 or epoch == n_epochs-1:
      validate(epoch)

In [ ]:
for epoch in range(1, n_epochs + 1):
    train(epoch)

Epoch: 1 | Loss: 0.7496: 100%|██████████| 669/669 [04:53<00:00,  2.28it/s]
Epoch: 2 | Loss: 0.7751:   0%|          | 0/669 [00:00<?, ?it/s]
  0%|          | 0/74 [00:00<?, ?it/s]
    Epoch: 2 | Validation Loss: 0.8197:   0%|          | 0/74 [00:00<?, ?it/s]
    Epoch: 2 | Validation Loss: 0.8197:   1%|▏         | 1/74 [00:00<00:20,  3.50it/s]
    Epoch: 2 | Validation Loss: 0.7420:   1%|▏         | 1/74 [00:00<00:20,  3.50it/s]
    Epoch: 2 | Validation Loss: 0.7420:   3%|▎         | 2/74 [00:00<00:12,  5.63it/s]
    Epoch: 2 | Validation Loss: 0.7626:   3%|▎         | 2/74 [00:00<00:12,  5.63it/s]
    Epoch: 2 | Validation Loss: 0.6138:   3%|▎         | 2/74 [00:00<00:12,  5.63it/s]
    Epoch: 2 | Validation Loss: 0.6138:   5%|▌         | 4/74 [00:00<00:08,  8.29it/s]
    Epoch: 2 | Validation Loss: 0.7130:   5%|▌         | 4/74 [00:00<00:08,  8.29it/s]
    Epoch: 2 | Validation Loss: 0.6551:   5%|▌         | 4/74 [00:00<00:08,  8.29it/s]
    Epoch: 2 | Validation Loss: 0.6551:   8%|▊

plots of the loss evolution

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses)
plt.legend(['Train Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
plt.show()

# 5. Test the resulting network on examples from an independent test set. Implement and present: (40 points)
a. Predictions for (μ, aleatoric, epistemic) .            
b. Visualizations for (μ, aleatoric, epistemic) on 5 different input examples.         
c. Comment briefly on how the model’s performance could be improved.          
d. Please save your code and results for submission.

# References
[1] https://blog.csdn.net/shwan_ma/article/details/100012808         
[2] https://blog.csdn.net/oYeZhou/article/details/112270908